In [ ]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [300]:
labels = ["O",
          "B_ORG", "B_PER", 
          "B_LOC", "B_MEA", 
          "I_DTM", "I_ORG", 
          "E_ORG", "I_PER", 
          "B_TTL", "E_PER", 
          "B_DES", "E_LOC", 
          "B_DTM", "B_NUM", 
          "I_MEA", "E_DTM", 
          "E_MEA", "I_LOC", 
          "I_DES", "E_DES", 
          "I_NUM", "E_NUM", 
          "B_TRM", "B_BRN", 
          "I_TRM", "E_TRM", 
          "I_TTL", "I_BRN", 
          "E_BRN", "E_TTL", "B_NAME"]

In [250]:
import os
import csv


def make_csv_for_text(input_folder, output_csv):
    with open(output_csv, mode="w", encoding="utf-8", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Filename", "words", "POS", "Entity", "Class"])
    
        for filename in sorted(os.listdir(input_folder)):
            if filename.endswith('.txt'):
                file_path = os.path.join(input_folder, filename)
    
                with open(file_path, mode="r", encoding="utf-8") as txt_file:
                    for line in txt_file:
                        parts = line.strip().split("\t")
                        if len(parts) > 1:
                            writer.writerow([filename] + parts)
    
    print(f"แปลงไฟล์ .txt ทั้งหมดในโฟลเดอร์ '{input_folder}' เป็น '{output_csv}' เสร็จสิ้น!")
    

In [301]:
make_csv_for_text(input_folder="train/train",output_csv="train.csv")
make_csv_for_text(input_folder="eval/eval",output_csv="eval.csv")
make_csv_for_text(input_folder="test/test",output_csv="test.csv")

แปลงไฟล์ .txt ทั้งหมดในโฟลเดอร์ 'train/train' เป็น 'train.csv' เสร็จสิ้น!
แปลงไฟล์ .txt ทั้งหมดในโฟลเดอร์ 'eval/eval' เป็น 'eval.csv' เสร็จสิ้น!
แปลงไฟล์ .txt ทั้งหมดในโฟลเดอร์ 'test/test' เป็น 'test.csv' เสร็จสิ้น!


In [343]:
import time
import pandas as pd

print("Loading data file now, this could take a while depending on file size")
start = time.time()
df_train = pd.read_csv('train.csv',index_col=False)
df_eval =  pd.read_csv('eval.csv',index_col=False)
df_test =  pd.read_csv('test.csv',index_col=False)
end = time.time()
print("Loading took " + str(round(end - start, 2)) + " seconds\n")


Loading data file now, this could take a while depending on file size
Loading took 1.64 seconds



In [344]:
import os
import glob
import pandas as pd
import re
from tqdm.auto import tqdm

In [345]:
from datasets import Dataset, DatasetDict

In [346]:
df_train.head()

,Filename,words,POS,Entity,Class
0,00001.txt,สภาสังคมสงเคราะห์แห่งประเทศ,NN,B_ORG,B_CLS
1,00001.txt,ไทย,NN,E_ORG,I_CLS
2,00001.txt,จี้,VV,O,I_CLS
3,00001.txt,ศาล,NN,O,I_CLS
4,00001.txt,ไฟเขียว,VV,O,I_CLS


In [347]:
import pandas as pd

# Load the tag list from the CSV file
tag_list_df = pd.read_csv('tag_list.csv')

# Create label2id and id2label dictionaries
label2id = {label: idx for idx, label in enumerate(tag_list_df['tag'])}
id2label = {idx: label for label, idx in label2id.items()}

num_labels=len(label2id)
print("label2id:", label2id)
print("id2label:", id2label)

label2id: {'O': 0, 'B_ORG': 1, 'B_PER': 2, 'B_LOC': 3, 'B_MEA': 4, 'I_DTM': 5, 'I_ORG': 6, 'E_ORG': 7, 'I_PER': 8, 'B_TTL': 9, 'E_PER': 10, 'B_DES': 11, 'E_LOC': 12, 'B_DTM': 13, 'B_NUM': 14, 'I_MEA': 15, 'E_DTM': 16, 'E_MEA': 17, 'I_LOC': 18, 'I_DES': 19, 'E_DES': 20, 'I_NUM': 21, 'E_NUM': 22, 'B_TRM': 23, 'B_BRN': 24, 'I_TRM': 25, 'E_TRM': 26, 'I_TTL': 27, 'I_BRN': 28, 'E_BRN': 29, 'E_TTL': 30, 'B_NAME': 31}
id2label: {0: 'O', 1: 'B_ORG', 2: 'B_PER', 3: 'B_LOC', 4: 'B_MEA', 5: 'I_DTM', 6: 'I_ORG', 7: 'E_ORG', 8: 'I_PER', 9: 'B_TTL', 10: 'E_PER', 11: 'B_DES', 12: 'E_LOC', 13: 'B_DTM', 14: 'B_NUM', 15: 'I_MEA', 16: 'E_DTM', 17: 'E_MEA', 18: 'I_LOC', 19: 'I_DES', 20: 'E_DES', 21: 'I_NUM', 22: 'E_NUM', 23: 'B_TRM', 24: 'B_BRN', 25: 'I_TRM', 26: 'E_TRM', 27: 'I_TTL', 28: 'I_BRN', 29: 'E_BRN', 30: 'E_TTL', 31: 'B_NAME'}


In [348]:
df_train['Entity'] = df_train['Entity'].apply(lambda x: x if x in labels else "O")
df_eval['Entity'] = df_eval['Entity'].apply(lambda x: x if x in labels else "O")

In [349]:
df_train['ner'] = df_train['Entity'].map(label2id)
df_eval['ner'] = df_eval['Entity'].map(label2id)

In [350]:
label2id_pos_dict = {label: idx for idx, label in enumerate(df_train['POS'].unique())}
id2label_pos_dict = {idx: label for label, idx in label2id_pos_dict.items()}
# Step 2: Map the POS column using the created dictionary

df_train['pos_tags'] = df_train['POS'].map(label2id_pos_dict)
df_eval['pos_tags'] = df_eval['POS'].map(label2id_pos_dict)
df_test['pos_tags'] = df_test['POS'].map(label2id_pos_dict)

print("label2id_pos_dict:", label2id_pos_dict)
print("\n")
print("id2label_pos_dict:", id2label_pos_dict)

label2id_pos_dict: {'NN': 0, 'VV': 1, 'PU': 2, 'CC': 3, 'PS': 4, 'NG': 5, 'AX': 6, 'NU': 7, 'AV': 8, 'AJ': 9, 'FX': 10, 'CL': 11, 'PR': 12, 'PA': 13, 'XX': 14, 'IJ': 15}


id2label_pos_dict: {0: 'NN', 1: 'VV', 2: 'PU', 3: 'CC', 4: 'PS', 5: 'NG', 6: 'AX', 7: 'NU', 8: 'AV', 9: 'AJ', 10: 'FX', 11: 'CL', 12: 'PR', 13: 'PA', 14: 'XX', 15: 'IJ'}


In [351]:
df_train

,Filename,words,POS,Entity,Class,ner,pos_tags
0,00001.txt,สภาสังคมสงเคราะห์แห่งประเทศ,NN,B_ORG,B_CLS,1,0
1,00001.txt,ไทย,NN,E_ORG,I_CLS,7,0
2,00001.txt,จี้,VV,O,I_CLS,0,1
3,00001.txt,ศาล,NN,O,I_CLS,0,0
4,00001.txt,ไฟเขียว,VV,O,I_CLS,0,1
...,...,...,...,...,...,...,...
2804216,03794.txt,ระหว่าง,PS,O,I_CLS,0,4
2804217,03794.txt,เรียน,VV,O,I_CLS,0,1
2804218,03794.txt,กับ,PS,O,I_CLS,0,4
2804219,03794.txt,งาน,NN,O,I_CLS,0,0


In [352]:
df_test['Class'] = df_test["Entity"]
df_test.drop(columns=["Entity"], inplace=True)
df_test.head()

,Filename,words,POS,Class,pos_tags
0,03795.txt,รัฐ,NN,B_CLS,0
1,03795.txt,ถังแตก,VV,I_CLS,1
2,03795.txt,วิก,NN,I_CLS,0
3,03795.txt,_,NN,I_CLS,0
4,03795.txt,7,NN,I_CLS,0


In [353]:
df_train.rename(columns={'Filename':'sentence_id'}, inplace=True)
df_eval.rename(columns={'Filename':'sentence_id'}, inplace=True)
df_test.rename(columns={'Filename':'sentence_id'}, inplace=True)

In [354]:
def new_id_sentence(df):
    new_sentence_ids = []
    sentence_counter = -1
    for index, row in df.iterrows():
        if row['Class'] == 'B_CLS':
            sentence_counter += 1
        new_sentence_ids.append(sentence_counter)
    df['sentence_id'] = new_sentence_ids
new_id_sentence(df_train)
new_id_sentence(df_eval)
new_id_sentence(df_test)

In [355]:
df_train.drop(columns=["Class"], inplace=True)
df_eval.drop(columns=["Class"], inplace=True)
df_test.drop(columns=["Class"], inplace=True)

In [356]:
from sklearn.preprocessing import LabelEncoder
label_encoder_ner = LabelEncoder()
label_encoder_pos = LabelEncoder()

df_train['encoded_ner'] = label_encoder_ner.fit_transform(df_train['ner'])
df_train['encoded_pos'] = label_encoder_pos.fit_transform(df_train['pos_tags'])

df_eval['encoded_ner'] = label_encoder_ner.fit_transform(df_eval['ner'])
df_eval['encoded_pos'] = label_encoder_pos.fit_transform(df_eval['pos_tags'])

In [362]:
df_train = df_train.drop(columns=['pos_tags'])

# เปลี่ยนชื่อคอลัมน์ให้เหมาะสม
df_train = df_train.rename(columns={
    'words': 'word',
    'Entity': 'entity',
    'ner': 'label'
})

label_encoder_ner = LabelEncoder()
df_train['encoded_labels'] = label_encoder_ner.fit_transform(df_train['label'])

In [363]:
df_eval = df_eval.drop(columns=['pos_tags'])

# เปลี่ยนชื่อคอลัมน์ให้เหมาะสม
df_eval = df_eval.rename(columns={
    'words': 'word',
    'Entity': 'entity',
    'ner': 'label'
})
label_encoder_ner = LabelEncoder()
df_eval['encoded_labels'] = label_encoder_ner.fit_transform(df_eval['label'])

In [364]:
words = df_train['word'].tolist()  # list ของคำ
labels = df_train['encoded_labels'].tolist()  # list ของ label (ตัวเลข)
entities = df_train['entity'].tolist()  # list ของ entity (เช่น B_ORG, O)

# แสดงตัวอย่างข้อมูล
print("Words:", words[:10])
print("Labels:", labels[:10])
print("Entities:", entities[:10])

Words: ['สภาสังคมสงเคราะห์แห่งประเทศ', 'ไทย', 'จี้', 'ศาล', 'ไฟเขียว', 'ขาย', 'สินทรัพย์', 'ปู', 'ทาง', 'ปรับ']
Labels: [1, 7, 0, 0, 0, 0, 0, 0, 0, 0]
Entities: ['B_ORG', 'E_ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [367]:
df_train = df_train.drop(columns=['sentence_id', 'POS', 'encoded_pos', 'encoded_labels'])
df_eval = df_eval.drop(columns=['sentence_id', 'POS', 'encoded_pos', 'encoded_labels'])

In [369]:
from transformers import BertTokenizer

# โหลด Tokenizer ที่รองรับภาษาไทย
tokenizer = BertTokenizer.from_pretrained('Geotrend/bert-base-th-cased')

In [379]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['word'], 
        truncation=True,  # ตัดข้อความให้ไม่ยาวเกินไป
        padding='max_length',  # เติม padding ถ้าจำเป็น
        max_length=512,  # กำหนดขนาดสูงสุด
        is_split_into_words=True
    )
    labels = examples['encoded_ner']
    new_labels = []
    
    for word, label in zip(examples['word'], labels):
        tokenized_word = tokenizer.tokenize(word)
        label_for_word = [label] * len(tokenized_word)  # ทุก token จะมี label เดียวกัน
        
        new_labels.extend(label_for_word)

    new_labels = new_labels[:512] + [-100] * (512 - len(new_labels))  # Padding ให้มีความยาว 512
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [373]:
# แปลงข้อมูลเป็น Dataset ของ Hugging Face
Train = Dataset.from_pandas(df_train)
Eval = Dataset.from_pandas(df_eval)


In [ ]:
Eval_tokenized_datasets = Eval.map(tokenize_and_align_labels, batched=True)
Train_tokenized_datasets = Train.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/248470 [00:00<?, ? examples/s]


ArrowInvalid: Column 4 named input_ids expected length 1000 but got length 512

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ฟังก์ชันสำหรับคำนวณ metrics (accuracy, precision, recall, F1 score)
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)  # แปลงจาก logits เป็น class ที่มีค่าสูงสุด
    
    # ใช้ precision_recall_fscore_support เพื่อคำนวณ Precision, Recall, F1, Accuracy
    precision, recall, f1, support = precision_recall_fscore_support(labels, predictions, average='weighted')
    
    # คำนวณ accuracy
    accuracy = accuracy_score(labels, predictions)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
from transformers import BertForTokenClassification, Trainer, TrainingArguments

# สร้างโมเดล NER
model = BertForTokenClassification.from_pretrained('Geotrend/bert-base-th-cased', num_labels=32)

# กำหนดการฝึก
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=5,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
)

# สร้าง Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Train_tokenized_datasets,
    eval_dataset=Eval_tokenized_datasets,
)

In [278]:
label_list = labels

In [279]:
from transformers import AutoTokenizer, AutoModelForTokenClassification , DataCollatorForTokenClassification

tokenizer = AutoTokenizer.from_pretrained(pretrained)
model = AutoModelForTokenClassification.from_pretrained(pretrained, num_labels=len(label_list),ignore_mismatched_sizes=True)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Geotrend/bert-base-th-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="BertForToken",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=10
)



d:\Anaconda\envs\NLP\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [52]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_Train,
    eval_dataset=tokenized_datasets_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\Acer\AppData\Local\Temp\ipykernel_64648\1061095081.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [53]:
trainer.train()

  0%|          | 0/1752640 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.